## Aula 01: Problema

### Encontre os valores de $a$, $b$, $c$, $d$, $e$ e %f% que satisfazem a equação
$$
a + 2b + 3c + 4d + 5e + 6f = 420
$$

In [1]:
import random
import numpy as np

In [2]:
def init(size):
    pop = []
    
    for i in range(size):
        a = random.randint(-30, 30)
        b = random.randint(-30, 30)
        c = random.randint(-30, 30)
        d = random.randint(-30, 30)
        e = random.randint(-30, 30)
        f = random.randint(-30, 30)
        
        solution = [a, b, c, d, e, f]
        
        pop.append(solution)
    
    return pop

In [3]:
def fitness(pop):
    fit = []
    obj = []
    
    for s in pop:
        v = (s[0] + 2*s[1] + 3*s[2] + 4*s[3] + 5*s[4] + 6*s[5]) - 420
        v = abs(v)
        f = 1 / (1 + v)
        
        obj.append(v)
        fit.append(f)
    
    return fit, obj

In [4]:
def elitism_selection(pop, fit, num_parents):
    sorted_pop = [p for _, p in sorted(zip(fit, pop), reverse=True)]
    parents = sorted_pop[:num_parents]
    
    return parents

In [5]:
def steady_selection(pop, fit, num_parents):
    sorted_pop = [p for _, p in sorted(zip(fit, pop), reverse=True)]
    prob = np.linspace(0.5, 0.1, len(pop))
    parents = random.choices(sorted_pop, weights=prob, k=num_parents)
    
    return parents

In [6]:
def tournament_selection(pop, fit, num_parents):
    parents = []
    
    for i in range(num_parents):
        f1 = random.choice(list(range(len(pop))))
        f2 = random.choice(list(range(len(pop))))
        
        if fit[f1] >= fit[f2]:
            parents.append(pop[f1])
        else:
            parents.append(pop[f2])
    
    return parents

In [7]:
def roulette_selection(pop, fit, num_parents):
    total_fit = sum(fit)
    prob = []
    
    for f in fit:
        p = f / total_fit
        
        prob.append(p)
    
    parents = random.choices(pop, weights=prob, k=num_parents)
    
    return parents

In [8]:
def crossover(parents):
    children = []
    
    for i in range(len(parents)):
        p1 = parents[i]
        p2 = parents[(i + 1) % len(parents)]
        
        cut_point = random.choice([0, 1, 2])
        child = p1[:cut_point] + p2[cut_point:]
        
        children.append(child)
    
    return children

In [9]:
def mutation(pop, rate=0.25):
    new_pop = []
    
    for p in pop:
        chance = random.uniform(0, 1)
        
        if chance <= rate:
            mutation_point = random.choice([0, 1, 2, 3, 4, 5])
            p[mutation_point] = random.randint(-30, 30)
            new_pop.append(p)
        else:
            new_pop.append(p)
    
    return new_pop

## Algoritmo Final

In [10]:
def genetic_algorithm(selection_strategy):
    num_generations = 1000
    pop_size = 10
    num_parents = 5
    best = 0
    final_generation = -1

    pop = init(pop_size)
    for i in range(num_generations):
        fit, obj = fitness(pop)

        try:
            best = obj.index(0)
            final_generation = i
            break
        except ValueError:
            pass
        
        parents = selection_strategy(pop, fit, num_parents)

        children = crossover(parents)
        pop = parents + children
        pop = mutation(pop)
    
    return pop[best], final_generation

In [11]:
best, final_generation = genetic_algorithm(elitism_selection)

if final_generation != -1:
    print(f"Melhor solução encontrada na geração {final_generation}: {best}")
else:
    print("O algoritmo chegou ao final das iterações, mas não encontrou a solução...")

Melhor solução encontrada na geração 139: [4, 27, 22, 27, 22, 13]


## Extra: existe melhor estratégia de seleção pra esse problema?

In [12]:
strategies = {
    "Elitismo" : elitism_selection,
    "Estável"  : steady_selection,
    "Torneio"  : tournament_selection,
    "Roleta"   : roulette_selection,
}

num_runs = 1000
dump = {
    "Elitismo" : [],
    "Estável"  : [],
    "Torneio"  : [],
    "Roleta"   : [],
}
for strategy, func in strategies.items():
    print(f"Executando {num_runs} execuções usando {strategy}...")
    
    for i in range(num_runs):
        _, final_generation = genetic_algorithm(func)
        dump[strategy].append(final_generation)
    
    print(f"{strategy} testado!")

Executando 1000 execuções usando Elitismo...
Elitismo testado!
Executando 1000 execuções usando Estável...
Estável testado!
Executando 1000 execuções usando Torneio...
Torneio testado!
Executando 1000 execuções usando Roleta...
Roleta testado!


In [13]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [14]:
sns.set()
plt.rcParams["figure.figsize"] = (12, 6)

In [15]:
df = pd.DataFrame(dump)

In [16]:
df.describe()

,Elitismo,Estável,Torneio,Roleta
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,131.240000,128.134000,105.660000,127.065000
std,150.285435,114.280778,91.929056,130.146805
min,-1.000000,0.000000,0.000000,-1.000000
25%,31.750000,49.750000,43.000000,41.000000
50%,74.000000,96.000000,80.000000,85.000000
75%,171.000000,170.250000,140.250000,167.000000
max,902.000000,859.000000,809.000000,966.000000


- Estratégias que apresentam valor mínimo -1 indicam que não alcançaram solução ótima em nenhuma das rodadas
- A média indica que o Torneio chega à solução antes que os demais, mas a mediana (50%) indica que o Elitismo separa melhor rodadas rápidas de rodadas lentas (problema: Elistimo apresentou rodadas que falharam)
- O desvio (std) também joga a favor do Torneio; um valor menor aqui indica menos variação em torno da média
- A Roleta, de maneira geral, foi a pior estratégia: apresentou falhas, valor de geração final quase no limite das rodadas, média e desvio altos